In [23]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [24]:
import numpy as np
def npz_load(npzfile):
    with np.load(npzfile) as file:
        data = file['x']
        labels = file['y']
        sampling_rate = file['fs']
    return data,labels,sampling_rate

In [25]:
def load_n_npzfiles(npzfiles):
    data = []
    labels = []
    fs = None
    for npz_file in npzfiles:
        t_data,t_labels,sampling_rate = npz_load(npz_file)
        if fs is None:
            fs = sampling_rate
        elif fs!= sampling_rate:
            raise Exception("Mismatch in Sampling Rate")
        data.append(t_data)
        labels.append(t_labels)
    data = np.vstack(data)
    labels = np.hstack(labels)
    return data,labels


In [26]:
import os
path="/content/drive/MyDrive/project/DATA"
files = os.listdir(path)
fs = []
for i in files:
    fs.append(os.path.join(path,i))
X,Y = load_n_npzfiles(fs)
print(X.shape)
X = X[:15000]
Y = Y[:15000]
X = X.reshape((X.shape[0], X.shape[1], 1))

(66669, 3000, 1)


In [27]:
Y.shape

(15000,)

In [28]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
def cnn_builder():
  '''function that sets the CNN layers from input to keras.Model'''

  input_length =  3000

  inputs = keras.Input(shape=(input_length,1))

  #left leg - short filters

  x = layers.Conv1D( filters=64, kernel_size=50, strides=6 ,
                    name = 'conv1')(inputs)
  x = layers.BatchNormalization()(x)
  x = layers.ReLU()(x)

  x = layers.MaxPool1D(pool_size=8, strides=8, name='max1')(x)

  x = layers.Dropout(rate=0.5  , name ='dropout1' )(x)

  x = layers.Conv1D(filters=128, kernel_size=8, strides=1,
                    name = 'conv2', padding = 'same' )(x)
  x = layers.BatchNormalization()(x)
  x = layers.ReLU()(x)

  x = layers.Conv1D(filters=128, kernel_size=8, strides=1,
                    name = 'conv3', padding = 'same' )(x)
  x = layers.BatchNormalization()(x)
  x = layers.ReLU()(x)

  x = layers.Conv1D(filters=128, kernel_size=8, strides=1,
                    name = 'conv4', padding = 'same')(x)
  x = layers.BatchNormalization()(x)
  x = layers.ReLU()(x)

  x = layers.MaxPool1D(pool_size=4, strides=4, name = 'max2')(x)
  x = layers.Flatten()(x)

  #right leg - longer filters
  x2 = layers.Conv1D( filters=64, kernel_size=400, strides=50 ,
                    name = 'conv1_2')(inputs)
  x2 = layers.BatchNormalization()(x2)
  x2 = layers.ReLU()(x2)

  x2 = layers.MaxPool1D(pool_size=4, strides=4, name='max1_2')(x2)

  x2 = layers.Dropout(rate=0.5  , name ='dropout1_2' )(x2)

  x2 = layers.Conv1D(filters=128, kernel_size=6, strides=1,
                    name = 'conv2_2', padding='same' )(x2)
  x2 = layers.BatchNormalization()(x2)
  x2 = layers.ReLU()(x2)

  x2 = layers.Conv1D(filters=128, kernel_size=6, strides=1,
                    name = 'conv3_2', padding='same' )(x2)
  x2 = layers.BatchNormalization()(x2)
  x2 = layers.ReLU()(x2)

  x2 = layers.Conv1D(filters=128, kernel_size=6, strides=1,
                    name = 'conv4_2', padding='same' )(x2)
  x2 = layers.BatchNormalization()(x2)
  x2 = layers.ReLU()(x2)

  x2 = layers.MaxPool1D(pool_size=2, strides=2, name = 'max2_2')(x2)

  x2 = layers.Flatten()(x2)
  #concatenate both legs
  concat = layers.Concatenate(name = 'concat')([x, x2])

  #final layer as softmax
  outputs = layers.Dense(5, activation=tf.nn.softmax, name = 'soft') (concat)

  model = keras.Model(inputs=inputs, outputs=outputs, name='sleep')

  return model

In [29]:
def lstm_builder():

  inputs = keras.Input(shape=(5,2688))

  x = layers.Dropout(rate=0.5  , name ='lstm_dropout1' )(inputs)

  x1 = Bidirectional(layers.LSTM(512, return_sequences=True)) (x)
  x1 = layers.Dropout(rate=0.5  , name ='lstm_dropout2' )(x1)

  x1 = Bidirectional(layers.LSTM(512)) (x1)
  x1 = layers.Dropout(rate=0.5  , name ='lstm_dropout3' )(x1)

  x2 = layers.Dense(1024)(x)
  x2 = layers.BatchNormalization()(x2)
  x2 = layers.ReLU(name = 'lstm_residual1')(x2)

  addition = layers.Add(name = 'lstm_concat1')([x1, x2])


  x3 = layers.BatchNormalization()(addition)
  x3 = layers.ReLU(name = 'lstm_residual2')(x3)

  x3 = layers.Dropout(rate=0.5  , name ='lstm_dropout4' )(addition)

  x3 = layers.Flatten()(x3)

  outputs = layers.Dense(5, activation=tf.nn.softmax, name = 'lstm_soft') (x3)

  model_lstm = keras.Model(inputs=inputs, outputs=outputs, name='lstmsleep')

  return model_lstm



In [30]:
from tensorflow.keras.layers import Bidirectional #we will be needing this layer
from sklearn.model_selection import StratifiedKFold
import time
import gc
from datetime import datetime as dt
# perform training and prediction for 20 stratified folds


skf = StratifiedKFold(n_splits=5) #10 - 20

cv_results = []
k = 0
for train_index,test_index in skf.split(X, Y):
    begin_time_loop = dt.now()
    print('Current time: {}'.format(str(begin_time_loop)))
    k = k+1
    X_train,X_test=X[train_index],X[test_index]
    y_train,y_test=Y[train_index],Y[test_index]
    model1 =cnn_builder()
    model1.compile(loss=keras.losses.SparseCategoricalCrossentropy(),
              optimizer=keras.optimizers.Adam(learning_rate=0.001),
              metrics=['accuracy'])
    model1.fit(X_train, y_train,
                    batch_size=100,
                    epochs=6) # 24
    #get feature vectors
    layer_name = 'concat'
    intermediate_layer_model = keras.Model(inputs=model1.input,
                                  outputs=model1.get_layer(layer_name).output)
    intermediate_output_train = intermediate_layer_model.predict(X_train)
    intermediate_output_test = intermediate_layer_model.predict(X_test)
    X_train_lstm = intermediate_output_train
    X_test_lstm = intermediate_output_test
    a = np.zeros((1, 2688))
    X_train_lstm = np.insert(X_train_lstm, 0, a, axis=0)
    X_train_lstm = np.insert(X_train_lstm, 0, a, axis=0)
    X_train_lstm = np.insert(X_train_lstm, -1, a, axis=0)
    X_train_lstm = np.insert(X_train_lstm, -1, a, axis=0)
    X_test_lstm = np.insert(X_test_lstm, 0, a, axis=0)
    X_test_lstm = np.insert(X_test_lstm, 0, a, axis=0)
    X_test_lstm = np.insert(X_test_lstm, -1, a, axis=0)
    X_test_lstm = np.insert(X_test_lstm, -1, a, axis=0)

    num_train_clips = len(X_train_lstm) - 4
    num_test_clips = len(X_test_lstm) - 4

    clip_length = 2688

    X_train_lstm2 = np.empty((num_train_clips, 5, clip_length))  # Initialize empty array for training clips
    for i in range(num_train_clips):
      j = i + 2
      X_train_lstm2[i] = np.array([X_train_lstm[j-2], X_train_lstm[j-1], X_train_lstm[j], X_train_lstm[j+1], X_train_lstm[j+2]])

    X_test_lstm2 = np.empty((num_test_clips, 5, clip_length))  # Initialize empty array for testing clips
    for i in range(num_test_clips):
      j = i + 2
      X_test_lstm2[i] = np.array([X_test_lstm[j-2], X_test_lstm[j-1], X_test_lstm[j], X_test_lstm[j+1], X_test_lstm[j+2]])
    #LSTM
    lstm = lstm_builder()
    lstm.compile(loss=keras.losses.SparseCategoricalCrossentropy(),
              optimizer=keras.optimizers.Adam(learning_rate=0.001),
              metrics=['accuracy'])
    lstm.fit(X_train_lstm2, y_train,
                      batch_size=100,
                      epochs=5, validation_data=(X_test_lstm2, y_test)) #
    test_scores = lstm.evaluate(X_test_lstm2, y_test, verbose=0)
    print('Test accuracy:', test_scores[1])
    y_prob = lstm.predict(X_test_lstm2)
    y_pred = y_prob.argmax(axis=-1)
    elapsed = dt.now() - begin_time_loop
    print('Fold {} took {}'.format(str(k), str(elapsed)))
    fold_result = {'fold': k, 'time': elapsed,'y_test': y_test, 'y_pred': y_pred, 'y_prob': y_prob,
                'accuracy': test_scores[1] }
    del X_test_lstm2, X_train_lstm2, intermediate_output_train, intermediate_output_test
    gc.collect()
    cv_results.append(fold_result)



Current time: 2024-04-05 14:05:38.691563
Epoch 1/6
120/120 [==============================] - 8s 17ms/step - loss: 0.9460 - accuracy: 0.6367
Epoch 2/6
120/120 [==============================] - 2s 16ms/step - loss: 0.6352 - accuracy: 0.7617
Epoch 3/6
120/120 [==============================] - 2s 16ms/step - loss: 0.5627 - accuracy: 0.7908
Epoch 4/6
120/120 [==============================] - 2s 16ms/step - loss: 0.5216 - accuracy: 0.8048
Epoch 5/6
120/120 [==============================] - 2s 17ms/step - loss: 0.5103 - accuracy: 0.8085
Epoch 6/6
94/94 [==============================] - 1s 3ms/step
Epoch 1/5
120/120 [==============================] - 15s 53ms/step - loss: 1.3008 - accuracy: 0.5464 - val_loss: 19.3869 - val_accuracy: 0.1920
Epoch 2/5
120/120 [==============================] - 4s 36ms/step - loss: 0.9806 - accuracy: 0.6469 - val_loss: 7.3334 - val_accuracy: 0.1917
Epoch 3/5
120/120 [==============================] - 4s 37ms/step - loss: 0.8867 - accuracy: 0.6704 - val_loss

In [31]:
model1.save('/content/drive/MyDrive/project/CNNLSTM.h5')
lstm.save('/content/drive/MyDrive/project/LSTM.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [32]:

x=[]
for i in cv_results:
  x.append(i['accuracy'])
np.mean(x)

0.6001999974250793

In [33]:
model1.save('/content/drive/MyDrive/project/cnnfor-lstm.h5')

In [34]:
lstm.save('/content/drive/MyDrive/project/lstm1.h5')

In [ ]:
pip install tensorflow

In [35]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np
from tensorflow.keras.layers import Bidirectional #we will be needing this layer
from sklearn.model_selection import StratifiedKFold
import time
import gc
from datetime import datetime as dt

nsplits =5
sk = StratifiedKFold(n_splits=nsplits)
for train_idx,test_idx in sk.split(X,Y):
  X_train,X_test = X[train_idx],X[test_idx]
  y_train,y_test= Y[train_idx],Y[test_idx]
  X_c_train,X_c_test = X_train,X_test
  y_c_train,y_c_test = y_train,y_test
  X_train = np.reshape(X_train, (-1, X_train.shape[1], 1))
  X_test = np.reshape(X_test, (-1, X_test.shape[1], 1))
  print(X_train.shape[0])

# Apply standardization
  scaler = StandardScaler()
  X_train = scaler.fit_transform(X_train.reshape((X_train.shape[0], -1))).reshape(X_train.shape)
  X_test = scaler.transform(X_test.reshape((X_test.shape[0], -1))).reshape(X_test.shape)

# Define the model
  model = tf.keras.Sequential([
    tf.keras.layers.Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], 1)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv1D(filters=32, kernel_size=3, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling1D(pool_size=2),
    tf.keras.layers.Dropout(rate=0.25),
    tf.keras.layers.Conv1D(filters=64, kernel_size=3, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv1D(filters=64, kernel_size=3, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling1D(pool_size=2),
    tf.keras.layers.Dropout(rate=0.25),
    tf.keras.layers.Conv1D(filters=128, kernel_size=3, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv1D(filters=128, kernel_size=3, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling1D(pool_size=2),
    tf.keras.layers.Dropout(rate=0.5),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(units=256, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(rate=0.5),
    tf.keras.layers.Dense(units=128, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(rate=0.5),
    tf.keras.layers.Dense(units=5, activation='softmax')
  ])

# Compile the model
  model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam(lr=0.001), metrics=['accuracy'])

# Train the model
  history = model.fit(X_train, y_train, batch_size=32, epochs=20, validation_data=(X_test, y_test))

# Evaluate the model on test data
  loss, accuracy = model.evaluate(X_test, y_test)
  print(f'Test Loss: {loss:.4f}')
  print(f'Test Accuracy: {accuracy:.4f}')

12000


Epoch 1/20
375/375 [==============================] - 20s 36ms/step - loss: 1.1227 - accuracy: 0.6142 - val_loss: 4.9745 - val_accuracy: 0.5060
Epoch 2/20
375/375 [==============================] - 14s 36ms/step - loss: 0.7746 - accuracy: 0.7152 - val_loss: 0.6466 - val_accuracy: 0.7530
Epoch 3/20
375/375 [==============================] - 13s 35ms/step - loss: 0.6732 - accuracy: 0.7455 - val_loss: 0.6765 - val_accuracy: 0.7420
Epoch 4/20
375/375 [==============================] - 13s 36ms/step - loss: 0.6216 - accuracy: 0.7691 - val_loss: 0.6930 - val_accuracy: 0.7417
Epoch 5/20
375/375 [==============================] - 13s 36ms/step - loss: 0.6038 - accuracy: 0.7705 - val_loss: 0.6745 - val_accuracy: 0.7457
Epoch 6/20
375/375 [==============================] - 13s 36ms/step - loss: 0.5447 - accuracy: 0.7948 - val_loss: 0.8853 - val_accuracy: 0.6867
Epoch 7/20
375/375 [==============================] - 13s 36ms/step - loss: 0.4982 - accuracy: 0.8152 - val_loss: 0.7185 - val_accuracy:

Epoch 1/20
375/375 [==============================] - 20s 37ms/step - loss: 1.1126 - accuracy: 0.6135 - val_loss: 3.1171 - val_accuracy: 0.4483
Epoch 2/20
375/375 [==============================] - 13s 35ms/step - loss: 0.7628 - accuracy: 0.7207 - val_loss: 0.8122 - val_accuracy: 0.6950
Epoch 3/20
375/375 [==============================] - 14s 36ms/step - loss: 0.6820 - accuracy: 0.7442 - val_loss: 0.8481 - val_accuracy: 0.6813
Epoch 4/20
375/375 [==============================] - 14s 36ms/step - loss: 0.6079 - accuracy: 0.7674 - val_loss: 0.8948 - val_accuracy: 0.6867
Epoch 5/20
375/375 [==============================] - 14s 36ms/step - loss: 0.5845 - accuracy: 0.7793 - val_loss: 0.9121 - val_accuracy: 0.6577
Epoch 6/20
375/375 [==============================] - 13s 36ms/step - loss: 0.5342 - accuracy: 0.8011 - val_loss: 0.8098 - val_accuracy: 0.6880
Epoch 7/20
375/375 [==============================] - 13s 35ms/step - loss: 0.4755 - accuracy: 0.8228 - val_loss: 0.8215 - val_accuracy:

Epoch 1/20
375/375 [==============================] - 19s 37ms/step - loss: 1.2023 - accuracy: 0.5832 - val_loss: 2.3600 - val_accuracy: 0.4250
Epoch 2/20
375/375 [==============================] - 13s 36ms/step - loss: 0.7894 - accuracy: 0.7081 - val_loss: 0.5652 - val_accuracy: 0.7810
Epoch 3/20
375/375 [==============================] - 13s 35ms/step - loss: 0.6558 - accuracy: 0.7522 - val_loss: 1.1341 - val_accuracy: 0.6587
Epoch 4/20
375/375 [==============================] - 13s 35ms/step - loss: 0.5906 - accuracy: 0.7775 - val_loss: 0.8000 - val_accuracy: 0.7007
Epoch 5/20
375/375 [==============================] - 13s 35ms/step - loss: 0.5645 - accuracy: 0.7837 - val_loss: 0.6933 - val_accuracy: 0.7377
Epoch 6/20
375/375 [==============================] - 13s 36ms/step - loss: 0.5135 - accuracy: 0.8103 - val_loss: 0.7225 - val_accuracy: 0.7283
Epoch 7/20
375/375 [==============================] - 13s 35ms/step - loss: 0.4813 - accuracy: 0.8179 - val_loss: 0.8021 - val_accuracy:

Epoch 1/20
375/375 [==============================] - 19s 36ms/step - loss: 1.1366 - accuracy: 0.6043 - val_loss: 3.2213 - val_accuracy: 0.5287
Epoch 2/20
375/375 [==============================] - 14s 36ms/step - loss: 0.7811 - accuracy: 0.7104 - val_loss: 0.8513 - val_accuracy: 0.6880
Epoch 3/20
375/375 [==============================] - 14s 36ms/step - loss: 0.6970 - accuracy: 0.7357 - val_loss: 0.9110 - val_accuracy: 0.6530
Epoch 4/20
375/375 [==============================] - 13s 35ms/step - loss: 0.6294 - accuracy: 0.7641 - val_loss: 0.7735 - val_accuracy: 0.7450
Epoch 5/20
375/375 [==============================] - 14s 36ms/step - loss: 0.5872 - accuracy: 0.7798 - val_loss: 0.6713 - val_accuracy: 0.7643
Epoch 6/20
375/375 [==============================] - 13s 34ms/step - loss: 0.5445 - accuracy: 0.7942 - val_loss: 0.7591 - val_accuracy: 0.7497
Epoch 7/20
375/375 [==============================] - 13s 34ms/step - loss: 0.4937 - accuracy: 0.8142 - val_loss: 0.9179 - val_accuracy:

Epoch 1/20
375/375 [==============================] - 20s 37ms/step - loss: 1.0137 - accuracy: 0.6473 - val_loss: 4.6284 - val_accuracy: 0.4030
Epoch 2/20
375/375 [==============================] - 13s 35ms/step - loss: 0.7040 - accuracy: 0.7427 - val_loss: 4.4055 - val_accuracy: 0.4767
Epoch 3/20
375/375 [==============================] - 13s 35ms/step - loss: 0.6352 - accuracy: 0.7615 - val_loss: 4.7223 - val_accuracy: 0.4867
Epoch 4/20
375/375 [==============================] - 13s 35ms/step - loss: 0.5669 - accuracy: 0.7876 - val_loss: 3.3102 - val_accuracy: 0.5307
Epoch 5/20
375/375 [==============================] - 13s 35ms/step - loss: 0.5383 - accuracy: 0.7999 - val_loss: 14.5410 - val_accuracy: 0.4913
Epoch 6/20
375/375 [==============================] - 13s 36ms/step - loss: 0.5591 - accuracy: 0.7903 - val_loss: 5.9328 - val_accuracy: 0.5127
Epoch 7/20
375/375 [==============================] - 13s 36ms/step - loss: 0.4930 - accuracy: 0.8152 - val_loss: 6.0839 - val_accuracy

In [36]:
model.save('/content/drive/MyDrive/project/CNN.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
